In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv

np.random.seed(0)

In [ ]:
def get_pos_vel(itime=0):
    """Return Measured Position and True Velocity.
        - State space model:
            * x_{k+1} = A x_{k} + w_{k}
            * z_{k} = H x_{k} + v_{k}
        - System Model:
            (1) Initial condition:
                x_pos_{0} = 0 [m]
                x_vel_{0} = 80 [m/s]
                w_{k} = N(0, 10)
                v_{k} = N(0, 10)
            (2) System condition:
                A (state transition model)              = [[1, dt]
                                                           [0,  1]]
                H (observation model)                   = [1, 0]
                Q (covariance of the process noise)     = [[1, 0],
                                                           [0, 3]]
                R (covariance of the observation noise) = 10
            (3) State space model:
                x_pos_{k+1} = x_pos_{k} + x_vel_{k} * dt
                x_vel_{k+1} = x_vel_{k} + w_{k}
                z_pos_{k} = x_pos_{k} + v_{k}
    """
    w_mean = 0
    w_std = np.sqrt(10)
    w = np.random.normal(w_mean, w_std)  # w: system noise.

    v_mean = 0
    v_std = np.sqrt(10)
    v = np.random.normal(v_mean, v_std)  # v: measurement noise.

    x_pos_0 = 0                                       # x_pos_0: initial position [m] 
    x_vel_0 = 80                                      # x_vel_0: initial velocity [m/s]
    x_pos = x_pos_0 + x_vel_0 * (itime * time_width)  # x_pos: true position. 
    x_vel = x_vel_0 + w                               # x_vel: true velocity.
    z_pos = x_pos + v                                 # z_pos: measured position (observable) 

    return z_pos, x_vel

In [ ]:
def kalman_filter(z_meas, x_esti, P):
    """Estimate voltage using a kalman filter."""
    # (1) Prediction.
    x_pred = A @ x_esti
    P_pred = A @ P @ A.T + Q

    # (2) Kalman Gain.
    K = P_pred @ H.T @ np.linalg.inv(H @ P_pred @ H.T + R)

    # (3) Eastimation.
    x_esti = x_pred + K @ (z_meas - H @ x_pred)

    # (4) Error Covariance.
    P = P_pred - K @ H @ P_pred

    return x_esti, P

In [ ]:
# Input parameters.
time_start = 0
time_end = 10
time_width = 0.1

In [ ]:
# Initialization for system model.
# Define matrix with rank 2 for matrix operation.
A = np.array([[1, time_width],
              [0, 1]])
H = np.array([[1, 0]])
Q = np.array([[1, 0],
              [0, 3]])
R = np.array([[10]])

# Initialization for estimation.
x_0 = np.array([0, 20])  # position and velocity
P_0 = 5 * np.eye(2)

In [ ]:
time = np.arange(time_start, time_end, time_width)
n_samples = len(time)

In [ ]:
z_pos_meas_save = np.zeros(n_samples)
x_vel_true_save = np.zeros(n_samples)
x_pos_esti_save = np.zeros(n_samples)
x_vel_esti_save = np.zeros(n_samples)

In [ ]:
x_esti, P = np.zeros(2), 0
for i in range(n_samples):
    z_pos_meas, x_vel_true = get_pos_vel(i)
    if i == 0:
        x_esti, P = x_0, P_0
    else:
        x_esti, P = kalman_filter(z_pos_meas, x_esti, P)

    z_pos_meas_save[i] = z_pos_meas
    x_vel_true_save[i] = x_vel_true
    x_pos_esti_save[i] = x_esti[0]
    x_vel_esti_save[i] = x_esti[1]

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,5))

plt.subplot(1, 2, 1)
plt.plot(time, x_pos_esti_save, 'bo-', label='Estimation (KF)')
plt.plot(time, z_pos_meas_save, 'r*--', label='Measurements')
plt.legend(loc='upper left')
plt.title('Position: Meas. v.s. Esti. (KF)')
plt.xlabel('Time [sec]')
plt.ylabel('Position [m]')

plt.subplot(1, 2, 2)
plt.plot(time, x_vel_true_save, 'r*--', label='True')
plt.plot(time, x_vel_esti_save, 'bo-', label='Estimation (KF)')
plt.legend(loc='lower right')
plt.title('Velocity: True v.s. Esti. (KF)')
plt.xlabel('Time [sec]')
plt.ylabel('Velocity [m/s]')
plt.savefig('png/pos2vel_kf.png')